# LCA of the CeRAS aircraft - Plots

This notebook provides the data and plots for the CeRAS case study presented in the article "A Comprehensive and Generic Life Cycle Assessment Tool for Overall Aircraft Design".

### Plot 1: Relative contributions to impacts

In [12]:
# Processes contributions for midpoints
fig = lca_plot(LCA_OUTPUT_FILE, 
               result_step = 'characterization', 
               filter_option = 'default', 
               filter_level = 0, 
               percent = True,
               exclude_methods=['ecosystem_quality', 'natural_resources', 'only_non-CO2'],
               exclude_methods_by_unit=['DALYs']
              )

# Update layout
fig.update_layout(title=None, width=1000, height=700, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=15), legend_traceorder="reversed")
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black', tickangle=-45, ticks="outside",)
fig.update_yaxes(title='Relative share', showline=True, linewidth=1, linecolor='black', gridcolor='grey', gridwidth=0.05)

# Rename impact methods
for idx in range(len(fig.data)):
    fig.data[idx].x = [s.split("<br>")[0] for s in fig.data[idx].x]
    #fig.data[idx].x = [s.split("<br>")[0].replace('climate change with non-CO2', 'xxx').replace('climate change', 'climate change (without non-CO2)').replace('xxx', 'climate change (with non-CO2)') for s in fig.data[idx].x]
    fig.data[idx].marker.line.width = 0

# Plot and save
fig.show()
#fig.write_image('ceras_relative_contribution_ref_midpoints.pdf')

In [13]:
# Processes contributions for endpoints
fig = lca_plot(LCA_OUTPUT_FILE, 
               result_step = 'characterization', 
               filter_option = 'default', 
               filter_level = 0, 
               percent = True,
               include_methods=['total'],
               exclude_methods=['only_non-CO2'],
              )

# Update layout
fig.update_layout(title=None, width=1000, height=920, paper_bgcolor='rgba(0,0,0,0)',  plot_bgcolor='rgba(0,0,0,0)', font=dict(size=20), legend_traceorder="reversed")
fig.update_xaxes(showline=True, linewidth=0.5, linecolor='black', tickangle=-45, ticks="outside",)
fig.update_yaxes(title='Relative share', showline=True, linewidth=1, linecolor='black', gridcolor='grey', gridwidth=0.05)

# Rename impact methods
for idx in range(len(fig.data)):
    fig.data[idx].x = [s.split("<br>")[0].replace('total ', '').replace('with non-CO2', '') for s in fig.data[idx].x]
    fig.data[idx].marker.line.width = 0

# Plot and save
fig.show()
#fig.write_image('ceras_relative_contribution_ref_endpoints.pdf')

### Plot 2: Midpoints to endpoints for the different scenarios

In [15]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FuncFormatter
import numpy as np
import collections
from fastoad.io import VariableIO
import pandas as pd

def lca_stacked_df(files_dict: dict, result_step: str = 'characterization', normalize: bool = False, threshold: float = 0.05, file_formatter=None, include_methods: list[str] = None, exclude_methods: list[str] = None):
    """
    Returns a stacked bar plot with contributions of each activity for each impact method.
    Each bar corresponds to one output file.
    If normalize, the highest stacked bar is set to 1.0 and others are scaled accordingly.
    """

    # Set up variables and constants
    palette = sns.color_palette("tab10")
    RESULTS_KEY = 'lca:' + result_step + ':'
    all_data = []

    ### Collect data from xml files and create dataframe
    for group, file_dict in files_dict.items():
        for subgroup, file_path in file_dict.items():
            data = []
            variables = VariableIO(file_path, file_formatter).read()
            for variable in variables.names():
                if RESULTS_KEY not in variable:  # variable is not an LCA score
                    continue
                #method_name = variable.split(RESULTS_KEY)[-1].split(":")[0]
                method_name = variable.split(RESULTS_KEY)[-1].split(":")[0] + ':' + variable.split(RESULTS_KEY)[-1].split(":")[1]
                if not any(subset in method_name for subset in include_methods):
                    continue
                if any(subset in method_name for subset in exclude_methods):
                    continue
                if len(variable.split(method_name + ':')) > 1:  # variable is a contribution to LCA score (e.g. life cycle stage) instead of total
                    continue
                value = variables[variable].value[0]
                data.append((group, subgroup, method_name, value))
            all_data.extend(data)

    df = pd.DataFrame(all_data, columns=['Group', 'File', 'Impact', 'Value'])
    df = df.pivot_table(index=['Group', 'File'], columns='Impact', values='Value', aggfunc='sum')
    
    ### Normalize w.r.t. to highest stacked bar
    if normalize == 'max':
        df = df.div(df.sum(axis=1).max())
    elif isinstance(normalize, tuple) and len(normalize) == 2:
        # Extract the sum for the selected ('Group', 'File') dataset
        ref_value = df.sum(axis=1).loc[normalize]
        # Normalize the entire dataframe by this reference value
        df = df.div(ref_value)

    ### Aggregate into 'Others' the impacts that are consistently small across all bars 
    grouped_contributions = df.sum(axis=1)  # Sum of all contributions for each result file
    small_contributions = df.div(grouped_contributions, axis=0) < threshold  # Identify small contributions within each result file
    small_in_all = small_contributions.all(axis=0)  # True only if small in all result files
    if small_in_all.any():
        df['Others'] = df.loc[:, small_in_all].sum(axis=1)  # Add 'Others' column with sum of all consistently small contributions
        small_in_all['Others'] = False
        df = df.drop(columns=df.columns[small_in_all])  # Drop the original small contributions

    return df

In [14]:
# Load files
kerosene_2040 = pth.join(WORK_FOLDER_PATH, "kerosene_2040.xml")
biofuel_products_2040 = pth.join(WORK_FOLDER_PATH, "biofuel_products_2040.xml")
biofuel_residues_2040 = pth.join(WORK_FOLDER_PATH, "biofuel_residues_2040.xml")
efuel_grid_2040 = pth.join(WORK_FOLDER_PATH, "efuel_grid_2040.xml")
efuel_solar_2040 = pth.join(WORK_FOLDER_PATH, "efuel_solar_2040.xml")
kerosene_2025 = pth.join(WORK_FOLDER_PATH, "kerosene_2025.xml")
biofuel_products_2025 = pth.join(WORK_FOLDER_PATH, "biofuel_products_2025.xml")
biofuel_residues_2025 = pth.join(WORK_FOLDER_PATH, "biofuel_residues_2025.xml")
efuel_grid_2025 = pth.join(WORK_FOLDER_PATH, "efuel_grid_2025.xml")
efuel_solar_2025 = pth.join(WORK_FOLDER_PATH, "efuel_solar_2025.xml")

# Plot
df = lca_stacked_df(
    {
        '(a) Kerosene': {'2025': kerosene_2025, '2040': kerosene_2040},
        '(b) Biofuel - products': {'2025': biofuel_products_2025, '2040': biofuel_products_2040},
        '(c) Biofuel - residues': {'2025': biofuel_residues_2025, '2040': biofuel_residues_2040},
        '(d) E-fuel - grid': {'2025': efuel_grid_2025, '2040': efuel_grid_2040},
        '(e) E-fuel - solar': {'2025': efuel_solar_2025, '2040': efuel_solar_2040}
        },
    normalize=None, #('(a) Kerosene', '2025'), 
    threshold = 0.04,  # 0.025
    #include_methods=['human_health'],
    #exclude_methods=['photochemical_oxidant_formation_human_health', 'total_human_health', 'with_non-CO2']
    include_methods=['ecosystem_quality'],
    exclude_methods=['total_ecosystem_quality', 'with_non-CO2']
)

# Save data
#df.to_csv('ceras_human_health.csv')
df.to_csv('ceras_ecosystem.csv')

In [ ]:
# Reorder impacts for better plot (e.g. to show climate change non-CO2 next to climate change CO2)
#new_order = [1, 2, 3, 0, 4, 5]  # New column order by index positions
new_order = [0, 2, 4, 1, 6, 3, 5, 7]
df = df.iloc[:, new_order]
df

## Payload-range diagram

Note: for the purpose of this study, the original `payload_range.py` from fast-oad-core file was modified to include `needed_block_fuel`, `diversion:fuel`, `holding:fuel` and `reserve:fuel` in the output grid.

In [34]:
PR_CONFIGURATION_FILE = pth.join(DATA_FOLDER_PATH, "payload_range.yml")

# Generate the inputs
#pr_input_file = oad.generate_inputs(PR_CONFIGURATION_FILE, LCA_OUTPUT_FILE, overwrite=True)
pr_input_file = pth.join(WORK_FOLDER_PATH, "payload_range_inputs.xml")
oad.variable_viewer(pr_input_file)

In [ ]:
# Evaluate payload-range data
eval_problem = oad.evaluate_problem(PR_CONFIGURATION_FILE, overwrite=True)

In [37]:
#PR_OUTPUT_FILE = eval_problem.output_file_path
PR_OUTPUT_FILE = pth.join(WORK_FOLDER_PATH, "payload_range_outputs.xml")
oad.variable_viewer(PR_OUTPUT_FILE)

In [ ]:
# Plot figure
fig = oad.payload_range_plot(
    PR_OUTPUT_FILE,
    name="Payload-Range Diagram",
    mission_name="SPP_study",
    variable_of_interest="specific_burned_fuel"
)
fig.show()

## Bonus: visualisation of the LCA model

FAST-OAD-LCA stores the parametric LCA model under the name `model` in the database `Foreground DB`. Once retrieved, it is possible to visualise the processes and exchanges involved in your model in the form of a process tree.

In [ ]:
import lca_algebraic as agb
from lca_modeller.gui.plots import process_tree

my_lca_model = agb.findActivity('model', db_name='Foreground DB')
process_tree(my_lca_model, foreground_only=True)

Alternatively, you can visualise it in the form of a table:

In [ ]:
from lca_modeller.helpers import list_processes
list_processes(my_lca_model)

Finally, you can display all the parameters involved in your LCA model. Three of these parameters are specific to prospective scenarios and generated automatically: `model`, `pathway`, and `year`. The first two must be set as options in the FAST-OAD configuration file, while the year can be entered directly in the input files.

In [30]:
agb.list_parameters()

## Plots for OAD-LCA article

In [84]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import FuncFormatter
import numpy as np
import collections
from fastoad.io import VariableIO
import pandas as pd

def lca_stacked_df(files_dict: dict, result_step: str = 'characterization', normalize: bool = False, threshold: float = 0.05, file_formatter=None, include_methods: list[str] = None, exclude_methods: list[str] = None):
    """
    Returns a stacked bar plot with contributions of each activity for each impact method.
    Each bar corresponds to one output file.
    If normalize, the highest stacked bar is set to 1.0 and others are scaled accordingly.
    """

    # Set up variables and constants
    palette = sns.color_palette("tab10")
    RESULTS_KEY = 'lca:' + result_step + ':'
    all_data = []

    ### Collect data from xml files and create dataframe
    for group, file_dict in files_dict.items():
        for subgroup, file_path in file_dict.items():
            data = []
            variables = VariableIO(file_path, file_formatter).read()
            for variable in variables.names():
                if RESULTS_KEY not in variable:  # variable is not an LCA score
                    continue
                #method_name = variable.split(RESULTS_KEY)[-1].split(":")[0]
                method_name = variable.split(RESULTS_KEY)[-1].split(":")[0] + ':' + variable.split(RESULTS_KEY)[-1].split(":")[1]
                if not any(subset in method_name for subset in include_methods):
                    continue
                if any(subset in method_name for subset in exclude_methods):
                    continue
                if len(variable.split(method_name + ':')) > 1:  # variable is a contribution to LCA score (e.g. life cycle stage) instead of total
                    continue
                value = variables[variable].value[0]
                data.append((group, subgroup, method_name, value))
            all_data.extend(data)

    df = pd.DataFrame(all_data, columns=['Group', 'File', 'Impact', 'Value'])
    df = df.pivot_table(index=['Group', 'File'], columns='Impact', values='Value', aggfunc='sum')
    
    ### Normalize w.r.t. to highest stacked bar
    if normalize == 'max':
        df = df.div(df.sum(axis=1).max())
    elif isinstance(normalize, tuple) and len(normalize) == 2:
        # Extract the sum for the selected ('Group', 'File') dataset
        ref_value = df.sum(axis=1).loc[normalize]
        # Normalize the entire dataframe by this reference value
        df = df.div(ref_value)

    ### Aggregate into 'Others' the impacts that are consistently small across all bars 
    grouped_contributions = df.sum(axis=1)  # Sum of all contributions for each result file
    small_contributions = df.div(grouped_contributions, axis=0) < threshold  # Identify small contributions within each result file
    small_in_all = small_contributions.all(axis=0)  # True only if small in all result files
    if small_in_all.any():
        df['Others'] = df.loc[:, small_in_all].sum(axis=1)  # Add 'Others' column with sum of all consistently small contributions
        small_in_all['Others'] = False
        df = df.drop(columns=df.columns[small_in_all])  # Drop the original small contributions

    return df

# Files
kerosene_2040 = pth.join(WORK_FOLDER_PATH, "kerosene_2040.xml")
biofuel_products_2040 = pth.join(WORK_FOLDER_PATH, "biofuel_products_2040.xml")
biofuel_residues_2040 = pth.join(WORK_FOLDER_PATH, "biofuel_residues_2040.xml")
efuel_grid_2040 = pth.join(WORK_FOLDER_PATH, "efuel_grid_2040.xml")
efuel_solar_2040 = pth.join(WORK_FOLDER_PATH, "efuel_solar_2040.xml")
kerosene_2025 = pth.join(WORK_FOLDER_PATH, "kerosene_2025.xml")
biofuel_products_2025 = pth.join(WORK_FOLDER_PATH, "biofuel_products_2025.xml")
biofuel_residues_2025 = pth.join(WORK_FOLDER_PATH, "biofuel_residues_2025.xml")
efuel_grid_2025 = pth.join(WORK_FOLDER_PATH, "efuel_grid_2025.xml")
efuel_solar_2025 = pth.join(WORK_FOLDER_PATH, "efuel_solar_2025.xml")

# Plot
df = lca_stacked_df(
    {
        '(a) Kerosene': {'2025': kerosene_2025, '2040': kerosene_2040},
        '(b) Biofuel - products': {'2025': biofuel_products_2025, '2040': biofuel_products_2040},
        '(c) Biofuel - residues': {'2025': biofuel_residues_2025, '2040': biofuel_residues_2040},
        '(d) E-fuel - grid': {'2025': efuel_grid_2025, '2040': efuel_grid_2040},
        '(e) E-fuel - solar': {'2025': efuel_solar_2025, '2040': efuel_solar_2040}
        },
    normalize=None, #('(a) Kerosene', '2025'), 
    threshold = 0.04,  # 0.025
    #include_methods=['human_health'],
    #exclude_methods=['photochemical_oxidant_formation_human_health', 'total_human_health', 'with_non-CO2']
    include_methods=['ecosystem_quality'],
    exclude_methods=['total_ecosystem_quality', 'with_non-CO2']
)

# Save data
#df.to_csv('ceras_human_health.csv')
df.to_csv('ceras_ecosystem.csv')

In [85]:
df

In [86]:
# Reorder impacts for better plot (e.g. to show climate change non-CO2 next to climate change CO2)
#new_order = [1, 2, 3, 0, 4, 5]  # New column order by index positions
new_order = [0, 2, 4, 1, 6, 3, 5, 7]
df = df.iloc[:, new_order]
df

In [87]:
### Plot grouped stacked bars
plt.style.use('bmh')

# Set up main plot
clusters = df.index.levels[0]  # number of groups
inter_graph = 0
maxi = np.max(np.sum(df, axis=1))
total_width = (len(df)+inter_graph*(len(clusters)-1))
fig = plt.figure(figsize=(total_width,6))
gridspec.GridSpec(1, total_width)
palette = sns.color_palette("tab10")
palette.insert(3, palette[3])  # Duplicate color from climate change CO2 to climate change Non-CO2
#palette.insert(7, palette[7])  # Duplicate color from climate change CO2 to climate change Non-CO2
hatches = [''] * len(df.index)
hatches[4] = '//'  # Set hatches for climate change non-co2

# Set up subplots
axes=[]
ax_position = 0
for cluster in clusters: # one subplot per group
    subset = df.loc[cluster]
    ax = subset.plot(kind="bar", stacked=True, width=0.8, ax=plt.subplot2grid((1,total_width), (0,ax_position), colspan=len(subset.index)), color=palette, alpha=0.8, edgecolor='0.2')
    axes.append(ax)
    ax.set_title(cluster, fontsize=11)
    ax.set_xlabel("")
    ax.set_ylim(0,maxi*1.1)
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))
    ax_position += len(subset.index)+inter_graph
    ax.tick_params(axis='x', rotation=0, labelsize=11)
    ax.set_facecolor('white')
    ax.set_axisbelow(True)

    # Apply hatches to the specific segment of the stacks
    for bar_group, hatch_pattern in zip(ax.containers, hatches[:len(subset.columns)]):
        for bar in bar_group:
            bar.set_hatch(hatch_pattern)

# Set y axis only on the left of left subplot and right of right subplot
for i in range(1,len(clusters)-1):
    axes[i].set_yticklabels("")
    axes[i].yaxis.set_ticks_position('none') 
axes[-1].yaxis.tick_right()
axes[-1].tick_params(axis='y', labelsize=11, direction="out")
axes[0].tick_params(axis='y', labelsize=11, direction="out")
#axes[0].set_ylabel("Human Health - DALYs/(pax.km)", fontsize=12)
axes[0].set_ylabel("Ecosystem Quality - species.yr/(pax.km)", fontsize=12)

# Set ticks notation
def sci_format(x,lim):
    return '{:.1e}'.format(x)
major_formatter = FuncFormatter(sci_format)
axes[-1].yaxis.set_major_formatter(major_formatter)
axes[0].yaxis.set_major_formatter(major_formatter)

# Set a single legend for all plots instead of one per subplot
for i in range(0,len(clusters)):
    axes[i].legend().set_visible(False)
entries = collections.OrderedDict()
for ax in axes:
    for handle, label in zip(*axes[0].get_legend_handles_labels()):
        # label_name = label.replace('_', ' ').title()
        #label_name = label.replace('human_health_with_non-CO2:', '')
        label_name = label.replace('human_health_only_non-CO2:climate_change', 'climate_change (non-CO2)')
        label_name = label_name.replace('ecosystem_quality_only_non-CO2:climate_change', 'climate_change (non-CO2)')
        label_name = label_name.replace('human_health', '')
        label_name = label_name.replace('ecosystem_quality', '')
        label_name = label_name.replace('terrestrial_ecosystems', '')
        label_name = label_name.replace('_', ' ').replace(':', '').title()
        entries[label_name] = handle
legend = fig.legend(
    entries.values(), entries.keys(),
    loc='lower center', bbox_to_anchor=(0.5, 0),
    ncol=2,
    fontsize=13,
    title='Midpoint Category',
    title_fontsize=14
)

# Set tight layout while keeping legend in the screen
bbox = legend.get_window_extent(fig.canvas.get_renderer()).transformed(fig.transFigure.inverted())
fig.tight_layout(rect=(0, bbox.y1, 1, 1), h_pad=0.5, w_pad=0.5)

# Show plot
plt.show()

# Save fig
#fig.savefig('ceras_human_health.pdf')
fig.savefig('ceras_ecosystem.pdf')

### Payload-range diagram for climate change

In [55]:
from fastoad.gui.analysis_and_plots import *
import lca_algebraic as agb
import os.path as pth
import matplotlib.pyplot as plt
import scipy.interpolate as interp
from scipy.interpolate import griddata
import brightway2 as bw
import plotly


def payload_range_plot_lca(
    aircraft_file_path: Union[str, PathLike],
    name="Payload-Range",
    mission_name="operational",
    variable_of_interest: str = None,
    variable_of_interest_legend: str = None,
    lca_params: dict = None,
    x_min: float = 0.0,
    y_min: float = 0.0,
    x_max: float = None,
    y_max: float = None,
    n_contours: int = 12
):
    """
    Returns a figure of the payload-range diagram.

    :param aircraft_file_path: path of data file
    :param name: name to give to the trace added to the figure
    :param mission_name: name of the mission present in the data file to be plotted.
    :param variable_of_interest: impact category to plot (LCA)
    :param variable_of_interest_legend: name to give to variable of interest in plot legend.
    :param lca_params: dict of LCA parameters values corresponding to the mission and data file
    :return: payload-range plot figure
    """
    variables = VariableIO(aircraft_file_path).read()

    ### Contour of the payload range
    range_ = np.asarray(variables[f"data:payload_range:{mission_name}:range"].value)
    range_ = convert_units(range_, "m", "NM")
    payload = np.asarray(variables[f"data:payload_range:{mission_name}:payload"].value)
    payload = convert_units(payload, "kg", "t")
    pr_contour = go.Scatter(
        x=range_[range_<3000],
        y=payload,
        mode="lines",
        line=dict(color="black", width=3),
        showlegend=False,
        name=name,
    )
    
    ### Create mask for a nice payload range
    range_mask = np.append(range_, (1.1 * max(range_), 1.1 * max(range_), 0))
    payload_mask = np.append(payload, (0, 1.1 * max(payload), 1.1 * max(payload)))
    pr_contour_mask = go.Scatter(
        x=range_mask,
        y=payload_mask,
        mode="lines",
        line=dict(color="white", width=3),
        showlegend=False,
        name=name,
        fill="toself",
        fillcolor="white",
    )
    
    ### Grid for the payload range
    range_grid = np.asarray(variables[f"data:payload_range:{mission_name}:grid:range"].value)
    payload_grid = np.asarray(
        variables[f"data:payload_range:{mission_name}:grid:payload"].value
    )

    # LCA calculation
    if isinstance(variable_of_interest, tuple):
        variable_of_interest_unit = bw.Method(variable_of_interest).metadata['unit']
        variable_of_interest_grid = agb.compute_impacts(
            agb.findActivity('model', db_name='Foreground DB'),
            variable_of_interest,
            **lca_params
        ).values.ravel()

    else: # probably not an lca category but a variable from fast-oad
        variable_of_interest_unit = variables[
            f"data:payload_range:{mission_name}:grid:{variable_of_interest}"
        ].units
        variable_of_interest_grid = np.asarray(
            variables[f"data:payload_range:{mission_name}:grid:{variable_of_interest}"].value
        )
    
    x = convert_units(range_grid, "m", "NM")
    y = convert_units(payload_grid, "kg", "t")
    z = variable_of_interest_grid

    # Convert to meshgrid and apply filtering
    #xi = np.linspace(x.min(), x.max(), 200)
    xi = np.linspace(340, x.max(), 300)
    yi = np.linspace(y.min(), y.max(), 300)
    Xi, Yi = np.meshgrid(xi, yi)
    zi = griddata((x, y), z, (xi[None, :], yi[:, None]), method='linear', rescale=True)
    
    #zi_smooth = gaussian_filter(zi, sigma=.5)  # Adjust sigma for smoothing
    #fig, ax = plt.subplots(figsize=(12, 6))
    #ax.contour(Xi, Yi, zi_smooth, levels=14, linewidths=0.5, colors='k')
    #cntr = ax.contourf(Xi, Yi, zi, levels=14, cmap="RdBu_r")
    #return fig

    # Add to plot
    pr_grid = go.Contour(
                x=xi,
                y=yi,
                z=zi,
                contours=dict(start=min(zi.flatten()), end=max(zi.flatten()), size=(max(z) - min(z)) / n_contours, showlabels = True),
                #contours=dict(start=1.4e-3, end=4.8e-3, size=(max(z) - min(z)) / n_contours, showlabels = True),
                colorbar=dict(
                    title=variable_of_interest_unit + '/(kg.km)',  #'/(pax.km)',
                    titleside="right",
                    #titlefont=dict(size=15, family="Arial, sans-serif"),
                    tickformat=".2e",
                ),
                colorscale="RdBu_r",
                contours_coloring="heatmap",
                opacity=0.95,
                line_smoothing=0.8,
                ncontours=n_contours,
                connectgaps=True
            )

    ### Create hatch overlay where non-CO2 effects are predominant
    #z2 = variable_of_interest_grid * 0.4  # Assuming you have a second grid
    #zi2 = griddata((x, y), z2, (xi[None, :], yi[:, None]), method='linear', rescale=True)
    #mask = zi > 2 * zi2
    #hatch_overlay = go.Contour(
    #    x=xi,
    #    y=yi,
    #    z=mask.astype(int),  # Convert mask to binary for contour plotting
    #    showscale=False,  # No colorbar for hatch
    #    colorscale=[[0, "rgba(0,0,0,0)"], [1, "rgba(0,0,0,0.5)"]],  # Transparent and shaded region
    #    contours=dict(start=0.5, end=0.5, size=1),
    #    opacity=0.5,  # Adjust hatch visibility
    #    line=dict(width=0),
    #    connectgaps=True
    #)

    ### Plot
    fig = go.Figure()
    fig.add_trace(pr_contour_mask)
    fig.add_trace(pr_contour)
    fig.add_trace(pr_grid)
    #fig.add_trace(hatch_overlay)
    #fig.add_trace(go.Scatter(x=x, y=y, hovertext=z, mode="markers"))  

    # Add point for reference mission
    fig.add_trace(go.Scatter(x=[500.0], y=[10.8864], mode="markers", marker_symbol='diamond', marker_color="black", marker_size=8,))
    fig.add_annotation(
        x=500.0,
        y=10.8864,
        ax=0,
        ay=115,
        text="Reference mission",
        showarrow=True,
        arrowhead=2,
        arrowsize=2,
        standoff=7,
        xanchor="center",
        bordercolor="black",
        borderwidth=1,
        borderpad=4,
        bgcolor="white",
    )

    # Update layout
    if not x_max:
        x_max = max(payload_mask)
    if not y_max:
        y_max = max(range_mask)
    
    fig.update_layout(
        xaxis_title="Range [NM]",
        yaxis_title="Payload [tonnes]",
        yaxis_range=[y_min, y_max],
        xaxis_range=[x_min, x_max],
        showlegend=False,
        height=500,
        width=900,
        #title={"text": name, "y": 0.9, "x": 0.5, "xanchor": "center", "yanchor": "top"},
        plot_bgcolor="white",
        font=dict(
            #family="Courier New, monospace",
            size=14,
        )
    )
    fig.update_xaxes(
        ticks="outside", 
        gridcolor='black', griddash='dash',
        showline=True, linecolor='black'
        )
    fig.update_yaxes(
        ticks="outside", tickvals=[val for val in np.arange(0, 20, 2.5)] + [round(max(payload),1)], 
        gridcolor='black', griddash='dash', 
        showline=True, linecolor='black'
    )

    return fig


# Payload range outputs
WORK_FOLDER_PATH = "workdir"
payload_range_outputs = pth.join(WORK_FOLDER_PATH,'payload_range_outputs.xml')
variables = VariableIO(payload_range_outputs).read()

# Assign values for LCA
#payload_150_PAX = 13608.0  #variables['data:weight:aircraft:max_payload'].value[0]
#load_factor = [payload / payload_150_PAX for payload in variables['data:payload_range:SPP_study:grid:payload'].value]
payload_mass = variables['data:payload_range:SPP_study:grid:payload'].value
main_route_distance = [dist / 1000 for dist in variables['data:payload_range:SPP_study:grid:range'].value]  # convert from meters to km
needed_block_fuel = variables['data:payload_range:SPP_study:grid:block_fuel'].value  # total fuel for main route + diversion + holding + reserve
diversion_fuel = variables['data:payload_range:SPP_study:grid:diversion_fuel'].value
holding_fuel = variables['data:payload_range:SPP_study:grid:holding_fuel'].value
reserve_fuel = variables['data:payload_range:SPP_study:grid:reserve_fuel'].value

lca_params = {
    'data__mission__SPP_study__main_route__distance': main_route_distance,
    'data__mission__SPP_study__needed_block_fuel': needed_block_fuel,
    'data__mission__SPP_study__diversion__fuel': diversion_fuel,
    'data__mission__SPP_study__holding__fuel': holding_fuel,
    'data__mission__SPP_study__reserve__fuel': reserve_fuel,
    'year': 2025,
    'model': 'remind',
    'pathway': 'SSP2_PkBudg1150',
    #'lca__parameters__load_factor': 90.72 / 0.8,  # load_factor,
    #'data__TLAR__NPAX': 1.0,  # 150,
    'data__mission__SPP_study__payload': payload_mass,
    'lca__parameters__lifetime_cycle_number': 35000.0,
    'lca__parameters__fuel_contrail_coefficient': 0.63,
    'lca__parameters__fossil_kerosene_share': 1.0,
    'lca__parameters__emission_index_co2_fuel': 3.16,
    'lca__parameters__emission_index_nox_kerosene': 0.01514,
    'lca__parameters__emission_index_sulfur_kerosene': 0.0012,
    'lca__parameters__emission_index_soot_kerosene': 3e-05,
    'lca__parameters__residues_biofuel_share': 0.0,
    'lca__parameters__emission_index_nox_alternative_fuel': 0.01514,
    'lca__parameters__emission_index_sulfur_alternative_fuel': 0.0,
    'lca__parameters__emission_index_soot_alternative_fuel': 6e-06,
    'lca__parameters__products_biofuel_share': 0.0,
    'lca__parameters__efuel_share': 0.0,
    'lca__parameters__elec_solar_share': 0.0,
    'data__weight__aircraft__OWE': 41990.170585323955
}

# Plot payload range diagram with LCA values
fig = payload_range_plot_lca(
    payload_range_outputs, 
    name="Payload-Range Diagram",
    mission_name="SPP_study",
    #variable_of_interest="specific_burned_fuel",
    variable_of_interest=('ReCiPe 2016 v1.03, midpoint (H)', 'climate change',  'global warming potential (GWP100)'),
    #variable_of_interest=('ReCiPe 2016 v1.03, endpoint (H)', 'total: ecosystem quality', 'ecosystem quality'),
    #variable_of_interest=('ReCiPe 2016 v1.03, endpoint (H)', 'total: human health', 'human health'),
    #variable_of_interest=('Custom methods', 'climate change (with non-CO2)', 'global warming potential (GWP100)'),
    #variable_of_interest=('Custom methods', 'total: ecosystem quality (with non-CO2)', 'ecosystem quality'),
    #variable_of_interest=('ReCiPe 2016 v1.03, endpoint (H)', 'total: natural resources', 'natural resources'),
    #variable_of_interest=('Custom methods', 'total: human health (with non-CO2)', 'human health'),
    lca_params=lca_params,
    x_min = 0.0,
    y_min = 0.0,
    x_max = 3100.0,
    y_max = 20.0,
    n_contours = 21
)
fig.show()
#plotly.io.write_image(fig, 'payload_range_CO2_nonCO2.pdf', format='pdf')